# Manual Review of Heads and Features

The manual annotations are intended to serve 2 roles: 1. to evaluate the accuracy of the head assignments on time phrases in the BHSA data, 2. to evaluate the time phrase structure in the BHSA dataset, and 3. to gain hands-on exposure to the kinds of time phrases in the dataset. This process consisted of comparing the selected heads against the surface text of the time phrase, and of reading the time phrases in the context of a verse when questions or anomalies arose. The annotation process consisted of marking a given time phrase as "g" for "good," "b" for "bad," and "?" for questionable cases. These classifications refer to both head assignments and internal structuring of the time phrases in the BHSA. The markings are often accompanied with notes: for bad or questionable entries the note explains what is wrong, for good entries the note might describe an interesting phenomenon, in some cases it might give a "light caution" about a given phrase. 

The outcome of this annotation process suggests that a custom database is necessary to consistently represent time phrases: there are many cases in BHSA time phrases where the phrase is cleft into 2 adjacent parts whereas in the majority of the data they are kept together. This is an inconsistency that should be solved. In other cases, the notion of "phrase" is not broad enough to encompass the full range of expressions that can mark time. For instance, several time phrases are split off from the infinitives they direct, in which the infinitive is an event. This is because the ETCBC method defines the infinitive event as a clause, operating at a different hierarchical level than the phrase; phrases are defined as strictly non-predicative. In the framework of Construction Grammar (Goldberg, *Constructions*, 1995, Croft, *Radical Construction Grammar*, 2001) preferred by this study, the link between the two forms is likewise considered a constructional object. Indeed, many phrases in this dataset refer to event-like nouns, which are from a syntactical perspective non-predicative; במות "in the death of" is a very frequent example, but other cases include ביום ישׁועה "in the day of salvation," which assumes a salvation event. If the difference between time phrases and time "clauses" is seen as merely an incremental difference rather than categorical, then one can apply a unified strategy in analyzing these cases. 

### Load TF / BHSA Data

In [22]:
import collections, random, csv
import pandas as pd
from IPython.display import display
from tf.fabric import Fabric
from tf.app import use
pd.set_option('display.max_colwidth', 0)  # configure DataFrame to show full notes with no truncation

locs = {'bhsa':'~/text-fabric-data/etcbc/bhsa/tf/c',
        'heads':'~/github/etcbc/heads/tf/c',
        'custom':'~/github/csl/time_collocations/data/tf'}

# load BHSA
TF = Fabric(locations=locs.values())
api = TF.load('''

vs vt pdp gloss lex typ number  prs
g_cons_utf8 nu mother st language ls rela
obj_prep sem_set head nhead
note function

''')

A = use('bhsa', api=api, hoist=globals(), silent=True)

This is Text-Fabric 7.7.3
Api reference : https://annotation.github.io/text-fabric/Api/Fabric/

121 features found and 4 ignored
  0.00s loading features ...
   |     0.00s Not enough info for structure in otext, structure functionality will not work
   |     0.18s B g_cons_utf8          from /Users/cody/text-fabric-data/etcbc/bhsa/tf/c
   |     0.12s B lex                  from /Users/cody/text-fabric-data/etcbc/bhsa/tf/c
   |     0.10s B vs                   from /Users/cody/text-fabric-data/etcbc/bhsa/tf/c
   |     0.10s B vt                   from /Users/cody/text-fabric-data/etcbc/bhsa/tf/c
   |     0.10s B pdp                  from /Users/cody/text-fabric-data/etcbc/bhsa/tf/c
   |     0.12s B gloss                from /Users/cody/text-fabric-data/etcbc/bhsa/tf/c
   |     0.17s B typ                  from /Users/cody/text-fabric-data/etcbc/bhsa/tf/c
   |     0.18s B number               from /Users/cody/text-fabric-data/etcbc/bhsa/tf/c
   |     0.09s B prs                  from /U

In [3]:
def tokenPhrase(phrasenode):
    '''Tokenizes a phrase with
    dot-separated words.
    input: phrase node number
    output: token string'''
    words = [(F.g_cons_utf8.v(w) if F.lex.v(w) != 'H' else 'ה') for w in L.d(phrasenode, 'word')]
    return '.'.join(words)

def tokenHeads(headslist):
    '''same as tokenPhrase but with list of head word nodes'''
    return '.'.join((F.g_cons_utf8.v(w) if F.lex.v(w) != 'H' else 'ה') for w in headslist)

## Phrase Tokens and Phrase Heads
This search counts all of the discrete time phrase tokens in Hebrew and gathers data about their heads. This data is exported to a spreadsheet for manual inspection. Per every token, a key of its heads is saved into a dictionary, linked to a list of phrase nodes. Tokens that have more than 1 head are suspicious, since the surface form is the same. All other tokens will be exported with their standard heads for inspection.

In [4]:
tp_heads = collections.defaultdict(lambda: collections.defaultdict(list))
tp_nheads = collections.defaultdict(lambda: collections.defaultdict(list))
tp_count = collections.Counter()

tps = A.search('''

phrase function=Time
/with/
    word language=Hebrew
/-/

''', shallow=True)

for tp in tps:
    token = tokenPhrase(tp) # tokenize surface of phrase
    heads_token = tokenHeads(E.head.t(tp)) # get heads
    nheads_token = tokenHeads(E.nhead.t(tp)) # get semantic heads
    tp_heads[token][heads_token].append(tp) # look at its heads
        
    # only populate nheads with PP phrases, since nhead feature for NP is exactly the same
    if F.typ.v(tp) == 'PP':
        tp_nheads[token][nheads_token].append(tp)
        
    tp_count[token] += 1
    
suspect_heads = [tp for tp in tp_heads if len(tp_heads[tp]) > 1]
suspect_nheads = [tp for tp in tp_nheads if len(tp_nheads[tp]) > 1]

print(f'total phrase tokens 2 head mappings: {len(tp_heads)}')
print(f'total phrase tokens 2 nhead mappings: {len(tp_nheads)}')
print(f'total suspect heads: {len(suspect_heads)}')
print(f'total suspect nheads {len(suspect_nheads)}')

  1.15s 3953 results
total phrase tokens 2 head mappings: 1164
total phrase tokens 2 nhead mappings: 888
total suspect heads: 0
total suspect nheads 0


### Compile Manual Inspection Spreadsheet

Two spreadsheets are exported: a `tp_heads.csv` and a `tp_nheads.csv`. Two versions are needed since `nhead` ("nominal" head, which is probably better-named as "semantic head") is different in cases of prepositional phrases. The `nhead` and `head` features on phrases thus have a lot of overlap, but are not identical. The `tp_nheads.csv` contains only prepositional phrases, since their `nhead` (an object of the prep.) is always different from the `head`.

After export, the files are opened in Numbers and manually annotated with a series of tags. It is then re-saved as the same .csv file. The data is exported below, and the results of the manual annotations are described in the subsequent section.

The column `token` contains a tokenized string of the entire phrase. The column `(n)heads_token` contains a dot-separated list of heads assigned to the phrase (e.g. ב.ב are two heads, both ב, for a single phrase). `ex` and `ex_ref` contain an example and example verse reference.

In [5]:
tp_heads_data = []
tp_nheads_data = []
data_header = ['token', '(n)heads_token', 'freq', 'mark', 'note', 'ex_ref', 'ex', 'ex_node', 'ex_verse']

for htp, nhtp in zip(tp_heads.keys(), tp_nheads.keys()):
    head = next(tp for tp in tp_heads[htp])
    nhead = next(tp for tp in tp_nheads[nhtp])
    head_ex = random.choice(tp_heads[htp][head])
    nhead_ex = random.choice(tp_nheads[nhtp][nhead])
    
    head_ref, nhead_ref = ['{} {}:{}'.format(*T.sectionFromNode(ex)) for ex in (head_ex, nhead_ex)]
    head_txt, nhead_txt = [T.text(ex) for ex in (head_ex, nhead_ex)]
    head_verse, nhead_verse = [T.text(L.u(ex, 'verse')[0]) for ex in (head_ex, nhead_ex)]
    
    heads_data = [htp, head, tp_count[htp], '', '', head_ref, head_txt, head_ex, head_verse]
    nheads_data = [nhtp, nhead, tp_count[nhtp], '', '', nhead_ref, nhead_txt, nhead_ex, nhead_verse]
    tp_heads_data.append(heads_data)
    tp_nheads_data.append(nheads_data)
    
tp_heads_data, tp_nheads_data = sorted(tp_heads_data), sorted(tp_nheads_data)

In [6]:
# !! Commented out to prevent writing over the manually added data !!

# with open('manual_curation/tp_heads.csv', 'w') as outfile:
#     writer = csv.writer(outfile)
#     writer.writerow(data_header)
#     writer.writerows(tp_heads_data)
    
# with open('manual_curation/tp_nheads.csv', 'w') as outfile:
#     writer = csv.writer(outfile)
#     writer.writerow(data_header)
#     writer.writerows(tp_nheads_data)

## Results of the Manual Inspection

In [10]:
head_anno = pd.read_csv('tp_heads.csv').fillna('')
nhead_anno = pd.read_csv('tp_nheads.csv').fillna('')

Cases are marked with one of three tags, `g` (good), `?` (questionable), or `b` (bad), based on whether the selected head/nhead is correct. The table below shows the breakdown of the manual annotations.

In [25]:
head_marks = pd.DataFrame.from_dict(collections.Counter(head_anno.mark), orient='index')
head_marks.index = ['good', 'questionable', 'bad']
head_marks['%'] = head_marks[0] / head_marks[0].sum()
head_marks = head_marks.sort_values(by=0, ascending=False)

nhead_marks = pd.DataFrame.from_dict(collections.Counter(nhead_anno.mark), orient='index')
nhead_marks.index = ['good', 'questionable', 'bad']
nhead_marks['%'] = nhead_marks[0] / nhead_marks[0].sum()
nhead_marks = nhead_marks.sort_values(by=0, ascending=False)

print('Head Marks')
display(head_marks)
print('')
print('N-Head Marks')
display(nhead_marks)

Head Marks


,0,%
good,1136,0.970111
questionable,21,0.017933
bad,14,0.011956



N-Head Marks


,0,%
good,802,0.897092
questionable,53,0.059284
bad,39,0.043624


97% of cases in head marks are rated good. N-heads shows a lower rate at 90%.

### Fixing Bad Cases

The bad cases are all exported to `head_fixes.csv` and `nhead_fixes.csv`. I add a new column, "fix", which contains a simple note proposing what solution should be adopted to remedy the bad case. Those solutions are broken down and explained below.

Export the spreadsheet for annotating proposed fixes to these problems.

**NB that the export is now commented out to prevent overwrite**

In [31]:
# head_anno[head_anno['mark'] == 'b'].to_csv('head_fixes.csv')

**The fixes are added and re-imported below.**

The solutions under 'fix' break down into the following:

* `function2` - these are cases which need to be fixed by reassigning the function of the phrase from `Time` to something else
* `define construction` - these are cases where a new object is needed to handle complexity above or below the phrase level
* `is good` - upon further inspection these cases are not erroneous

For particular cases there is additional information contained in the "note" column about the specific problem.

In [40]:
headfixes = pd.read_csv('head_fixes.csv').fillna('')
headfixes

,Unnamed: 0,token,(n)heads_token,freq,mark,note,fix,ex_ref,ex,ex_node,ex_verse
0,214,ב.ה.שׁמים.ו.ב.ה.ארץ,ב.ב,1,b,bad phrase function (not a time phrase); good heads,function2,Psalms 113:6,בַּשָּׁמַ֥יִם וּבָאָֽרֶץ׃,849296,הַֽמַּשְׁפִּילִ֥י לִרְאֹ֑ות בַּשָּׁמַ֥יִם וּבָאָֽרֶץ׃
1,251,ב.הר.ציון.ו.ב.ירושׁלם,ב.ב,1,b,not a time phrase; heads are good though,function2,Joel 3:5,בְּהַר־צִיֹּ֨ון וּבִירוּשָׁלִַ֜ם,825329,וְהָיָ֗ה כֹּ֧ל אֲשֶׁר־יִקְרָ֛א בְּשֵׁ֥ם יְהוָ֖ה יִמָּלֵ֑ט כִּ֠י בְּהַר־צִיֹּ֨ון וּבִירוּשָׁלִַ֜ם תִּֽהְיֶ֣ה פְלֵיטָ֗ה כַּֽאֲשֶׁר֙ אָמַ֣ר יְהוָ֔ה וּבַ֨שְּׂרִידִ֔ים אֲשֶׁ֥ר יְהוָ֖ה קֹרֵֽא׃
2,710,כל.ימי.כורשׁ.מלך.פרס.ו.עד.מלכות.דריושׁ.מלך.פרס,ימי.מלכות,1,b,"This TP should contain an NP specified by a PP with waw connection. The heads algorithm appears to have selected past the prep in this case. That is the expected behavior, but it has a bad effect here. There may be other cases like this, which could be isolated by finding independent preps within NP phrases and checking their head assignments.",define construction,Ezra 4:5,כָּל־יְמֵ֗י כֹּ֚ורֶשׁ מֶ֣לֶךְ פָּרַ֔ס וְעַד־מַלְכ֖וּת דָּרְיָ֥וֶשׁ מֶֽלֶךְ־פָּרָֽס׃,881852,וְסֹכְרִ֧ים עֲלֵיהֶ֛ם יֹועֲצִ֖ים לְהָפֵ֣ר עֲצָתָ֑ם כָּל־יְמֵ֗י כֹּ֚ורֶשׁ מֶ֣לֶךְ פָּרַ֔ס וְעַד־מַלְכ֖וּת דָּרְיָ֥וֶשׁ מֶֽלֶךְ־פָּרָֽס׃
3,858,מ.ה.יום.ה.הוא.ו.הלאה,מ,1,b,This TP misses והלאה; my suspicion is that these words are being marked as spec or atr when they should be para.,define construction,1_Samuel 18:9,מֵהַיֹּ֥ום הַה֖וּא וָהָֽלְאָה׃ ס,738491,וַיְהִ֥י שָׁא֖וּל עֹויֵ֣ן אֶת־דָּוִ֑ד מֵהַיֹּ֥ום הַה֖וּא וָהָֽלְאָה׃ ס
4,859,מ.ה.יום.ה.הוא.ו.מעלה,מ,2,b,"This is like הלאה but with מעלה; m. should be picked up as a coordinated noun. To prove this, it would be helpful to show comparable examples with waw connections.",define construction,1_Samuel 30:25,מֵֽהַיֹּ֥ום הַה֖וּא וָמָ֑עְלָה,743564,וַיְהִ֕י מֵֽהַיֹּ֥ום הַה֖וּא וָמָ֑עְלָה וַיְשִׂמֶ֜הָ לְחֹ֤ק וּלְמִשְׁפָּט֙ לְיִשְׂרָאֵ֔ל עַ֖ד הַיֹּ֥ום הַזֶּֽה׃ פ
5,865,מ.יום.ה.שׁמיני.ו.הלאה,מ,1,b,another case of הלאה not encoded,define construction,Leviticus 22:27,מִיֹּ֤ום הַשְּׁמִינִי֙ וָהָ֔לְאָה,690126,שֹׁ֣ור אֹו־כֶ֤שֶׂב אֹו־עֵז֙ כִּ֣י יִוָּלֵ֔ד וְהָיָ֛ה שִׁבְעַ֥ת יָמִ֖ים תַּ֣חַת אִמֹּ֑ו וּמִיֹּ֤ום הַשְּׁמִינִי֙ וָהָ֔לְאָה יֵרָצֶ֕ה לְקָרְבַּ֥ן אִשֶּׁ֖ה לַיהוָֽה׃
6,949,מהלך.יום.אחד,מהלך,1,b,"“a walk of one day”—this does not seem to be temporal but rather distance expressed in terms of time. This function should be adjusted, probably to Adju, but possibly Loca.",function2,Jonah 3:4,מַהֲלַ֖ךְ יֹ֣ום אֶחָ֑ד,828081,וַיָּ֤חֶל יֹונָה֙ לָבֹ֣וא בָעִ֔יר מַהֲלַ֖ךְ יֹ֣ום אֶחָ֑ד וַיִּקְרָא֙ וַיֹּאמַ֔ר עֹ֚וד אַרְבָּעִ֣ים יֹ֔ום וְנִֽינְוֵ֖ה נֶהְפָּֽכֶת׃
7,960,מן.ה.יום.ה.זה,מן,1,b,missing הלאה,is good,Haggai 2:19,מִן־הַיֹּ֥ום הַזֶּ֖ה,831777,הַעֹ֤וד הַזֶּ֨רַע֙ בַּמְּגוּרָ֔ה וְעַד־הַגֶּ֨פֶן וְהַתְּאֵנָ֧ה וְהָרִמֹּ֛ון וְעֵ֥ץ הַזַּ֖יִת לֹ֣א נָשָׂ֑א מִן־הַיֹּ֥ום הַזֶּ֖ה אֲבָרֵֽךְ׃ ס
8,961,מן.ה.יום.ה.זה.ו.מעלה,מן,1,b,missing ומעלה,define construction,Haggai 2:15,מִן־הַיֹּ֥ום הַזֶּ֖ה וָמָ֑עְלָה,831733,וְעַתָּה֙ שִֽׂימוּ־נָ֣א לְבַבְכֶ֔ם מִן־הַיֹּ֥ום הַזֶּ֖ה וָמָ֑עְלָה מִטֶּ֧רֶם שֽׂוּם־אֶ֛בֶן אֶל־אֶ֖בֶן בְּהֵיכַ֥ל יְהוָֽה׃
9,962,מן.ה.יום.ה.זה.ו.מעלה.מ.יום.עשׂרים.ו.ארבעה.ל.ה.תשׁיעי.ל.מן.ה.יום,מן.ל,1,b,"missing מעלה. It’s also not clear why second ל is picked up as a head. It is not necessarily NOT a head, but this TP seems to have some complex syntax. Investigation needed.",define construction,Haggai 2:18,מִן־הַיֹּ֥ום הַזֶּ֖ה וָמָ֑עְלָה מִיֹּום֩ עֶשְׂרִ֨ים וְאַרְבָּעָ֜ה לַתְּשִׁיעִ֗י לְמִן־הַיֹּ֛ום,831763,שִׂימוּ־נָ֣א לְבַבְכֶ֔ם מִן־הַיֹּ֥ום הַזֶּ֖ה וָמָ֑עְלָה מִיֹּום֩ עֶשְׂרִ֨ים וְאַרְבָּעָ֜ה לַתְּשִׁיעִ֗י לְמִן־הַיֹּ֛ום אֲשֶׁר־יֻסַּ֥ד הֵֽיכַל־יְהוָ֖ה שִׂ֥ימוּ לְבַבְכֶֽם׃


Now the nhead problems...

Export a spreadsheet for annotating proposed fixes to these problems.

**NB that the export is now commented out to prevent overwrite**

In [32]:
# nhead_anno[nhead_anno['mark'] == 'b'].to_csv('nhead_fixes.csv')

**The fixes are added and re-imported below.**

There is a new "fix" tag:

* head fixed - this issue has now been solved by re-running the [heads pipeline](https://github.com/ETCBC/heads) with corrections/adjustments

All other tags are identical in meaning to those found in `headfixes`.

In [39]:
nheadfixes = pd.read_csv('nhead_fixes.csv').fillna('')
nheadfixes

,Unnamed: 0,token,(n)heads_token,freq,mark,note,fix,ex_ref,ex,ex_node,ex_verse
0,10,אחרי,,4,b,no nhead,head fixed,Nehemiah 3:30,אַחֲרָ֨יו,884413,אַחֲרָ֨יו הֶחֱזִ֜יק חֲנַנְיָ֣ה בֶן־שֶׁלֶמְיָ֗ה וְחָנ֧וּן בֶּן־צָלָ֛ף הַשִּׁשִּׁ֖י מִדָּ֣ה שֵׁנִ֑י ס אַחֲרָ֣יו הֶחֱזִ֗יק מְשֻׁלָּם֙ בֶּן־בֶּ֣רֶכְיָ֔ה נֶ֖גֶד נִשְׁכָּתֹֽו׃ ס
1,38,אחריה,,1,b,no nhead,head fixed,2_Chronicles 11:20,אַחֲרֶ֣יהָ,897487,וְאַחֲרֶ֣יהָ לָקַ֔ח אֶֽת־מַעֲכָ֖ה בַּת־אַבְשָׁלֹ֑ום וַתֵּ֣לֶד לֹ֗ו אֶת־אֲבִיָּה֙ וְאֶת־עַתַּ֔י וְאֶת־זִיזָ֖א וְאֶת־שְׁלֹמִֽית׃
2,39,אחריהם,,2,b,no nhead,head fixed,Genesis 48:6,אַחֲרֵיהֶ֖ם,668648,וּמֹולַדְתְּךָ֛ אֲשֶׁר־הֹולַ֥דְתָּ אַחֲרֵיהֶ֖ם לְךָ֣ יִהְי֑וּ עַ֣ל שֵׁ֧ם אֲחֵיהֶ֛ם יִקָּרְא֖וּ בְּנַחֲלָתָֽם׃
3,40,אחריהן,,2,b,no nhead,head fixed,Genesis 41:27,אַחֲרֵיהֶ֗ן,665677,וְשֶׁ֣בַע הַ֠פָּרֹות הָֽרַקֹּ֨ות וְהָרָעֹ֜ת הָעֹלֹ֣ת אַחֲרֵיהֶ֗ן שֶׁ֤בַע שָׁנִים֙ הֵ֔נָּה וְשֶׁ֤בַע הַֽשִׁבֳּלִים֙ הָרֵקֹ֔ות שְׁדֻפֹ֖ות הַקָּדִ֑ים יִהְי֕וּ שֶׁ֖בַע שְׁנֵ֥י רָעָֽב׃
4,41,אחריו,,22,b,no nhead,head fixed,Nehemiah 3:22,אַחֲרָ֛יו,884359,וְאַחֲרָ֛יו הֶחֱזִ֥יקוּ הַכֹּהֲנִ֖ים אַנְשֵׁ֥י הַכִּכָּֽר׃
5,42,אחריך,,2,b,no nhead,head fixed,1_Kings 3:12,אַחֲרֶ֖יךָ,754959,הִנֵּ֥ה עָשִׂ֖יתִי כִּדְבָרֶ֑יךָ הִנֵּ֣ה׀ נָתַ֣תִּי לְךָ֗ לֵ֚ב חָכָ֣ם וְנָבֹ֔ון אֲשֶׁ֤ר כָּמֹ֨וךָ֙ לֹא־הָיָ֣ה לְפָנֶ֔יךָ וְאַחֲרֶ֖יךָ לֹא־יָק֥וּם כָּמֹֽוךָ׃
6,59,ב.אחרית.מלכותם,אחרית,1,b,should be מלכותם,head fixed,Daniel 8:23,בְאַחֲרִית֙ מַלְכוּתָ֔ם,879774,וּֽבְאַחֲרִית֙ מַלְכוּתָ֔ם כְּהָתֵ֖ם הַפֹּשְׁעִ֑ים יַעֲמֹ֛ד מֶ֥לֶךְ עַז־פָּנִ֖ים וּמֵבִ֥ין חִידֹֽות׃
7,113,ב.ה.חמשׁי,חמשׁי,1,b,this TP should be subsumed under the one immediately preceding it; the phrase boundary needs to be redrawn actually. Should pull comparable examples to validate this choice.,phrase2,Ezekiel 20:1,בַּֽחֲמִשִׁי֙,812472,וַיְהִ֣י׀ בַּשָּׁנָ֣ה הַשְּׁבִיעִ֗ית בַּֽחֲמִשִׁי֙ בֶּעָשֹׂ֣ור לַחֹ֔דֶשׁ בָּ֧אוּ אֲנָשִׁ֛ים מִזִּקְנֵ֥י יִשְׂרָאֵ֖ל לִדְרֹ֣שׁ אֶת־יְהוָ֑ה וַיֵּשְׁב֖וּ לְפָנָֽי׃ ס
8,119,ב.ה.יום.ה.הוא.ב.יום,יום,1,b,"The second part of this TP, ביום, is separated from an infinitival clause that is part of it. The whole part is an appositional use. I should consider redrawing this TP, perhaps to include the inf. clause. Or I should consider creating a construction object that it can be subsumed under.",define construction,Ezekiel 38:18,בַּיֹּ֣ום הַה֗וּא בְּיֹ֨ום,819264,וְהָיָ֣ה׀ בַּיֹּ֣ום הַה֗וּא בְּיֹ֨ום בֹּ֥וא גֹוג֙ עַל־אַדְמַ֣ת יִשְׂרָאֵ֔ל נְאֻ֖ם אֲדֹנָ֣י יְהוִ֑ה תַּעֲלֶ֥ה חֲמָתִ֖י בְּאַפִּֽי׃
9,134,ב.ה.יום.ה.שׁמיני.ו.הלאה,יום,1,b,missing הלאה,define construction,Ezekiel 43:27,בַיֹּ֨ום הַשְּׁמִינִ֜י וָהָ֗לְאָה,820983,וִֽיכַלּ֖וּ אֶת־הַיָּמִ֑ים ס וְהָיָה֩ בַיֹּ֨ום הַשְּׁמִינִ֜י וָהָ֗לְאָה יַעֲשׂ֨וּ הַכֹּהֲנִ֤ים עַל־הַמִּזְבֵּ֨חַ֙ אֶת־עֹולֹֽותֵיכֶם֙ וְאֶת־שַׁלְמֵיכֶ֔ם וְרָצִ֣אתִי אֶתְכֶ֔ם נְאֻ֖ם אֲדֹנָ֥י יְהֹוִֽה׃ ס


## Discussion on Fixes

There are a lot of cases marked as `define construction`. These can be broken down into N main groups:

* missing מעלה/הלאה and equivalents.
* quantified שׁנת "year of" phrases wherein the construct year is followed by a number and another instance of שׁנה. These are not properly missing heads cases, and could potentially be renamed as "cautions" rather than "bad." These particular cases will be handled in [chunking](../pipeline/chunking.py) which is run in the [pipeline](../pipeline/pipeline.ipynb). They are managed by treating the entire quantified phrase as a quant chunk object. That then allows the analysis to examine the quant chunk in its entirety in relation to שׁנת.
* מן...עד and מן...ועד. These are related to the first cases identified above. These cases are interesting because those with the waw should probably be marked as proper heads. But they are very similar to those without waw connections. The situation is perhaps similar to other cases when the BHSA marks an item as "parallel" (i.e. an independent element co-occurring with another independent element) but is not coordinated by waw. **These may represent cases that ought to be corrected in the heads pipeline**. At the same time, they are semantically determined. In other words, this problem might indeed be specific to this pattern. I need to do further work to find out the most consistent way to handle these cases. In the next stage, I will be analyzing complex time adverbial constructions, i.e. those such as these cases which have more than one preposition or time word. That stage will require that these cases be fully accounted for.
* the case in Ezra 4:5 (phrase 881852) where מלכות is selected instead of עד may represent a problem in the heads pipeline as well. This is a case where the phrase as a whole is a NP that contains a coordinate PP. The PP is marked as a separate phrase atom contained in the whole NP. **This case may be systemic across the heads dataset. I need to investigate what can be done to solve this in the heads pipeline.** 

### Looking at Questionable Cases

In [11]:
head_anno[head_anno['mark'] == '?']

,token,(n)heads_token,freq,mark,note,ex_ref,ex,ex_node,ex_verse
13,אחרי,אחרי,4,?,One case in Job with 1cs suffix אחרי that looks debatable,Nehemiah 3:31,אַחֲרָ֣יו,884421,אַחֲרָ֣יו הֶחֱזִ֗יק מַלְכִּיָּה֙ בֶּן־הַצֹּ֣רְפִ֔י עַד־בֵּ֥ית הַנְּתִינִ֖ים וְהָרֹכְלִ֑ים נֶ֚גֶד שַׁ֣עַר הַמִּפְקָ֔ד וְעַ֖ד עֲלִיַּ֥ת הַפִּנָּֽה׃
159,ב.ה.יום.ה.שׁמיני.ו.הלאה,ב,1,?,shouldn’t הלאה be considered a co-head here?,Ezekiel 43:27,בַיֹּ֨ום הַשְּׁמִינִ֜י וָהָ֗לְאָה,820983,וִֽיכַלּ֖וּ אֶת־הַיָּמִ֑ים ס וְהָיָה֩ בַיֹּ֨ום הַשְּׁמִינִ֜י וָהָ֗לְאָה יַעֲשׂ֨וּ הַכֹּהֲנִ֤ים עַל־הַמִּזְבֵּ֨חַ֙ אֶת־עֹולֹֽותֵיכֶם֙ וְאֶת־שַׁלְמֵיכֶ֔ם וְרָצִ֣אתִי אֶתְכֶ֔ם נְאֻ֖ם אֲדֹנָ֥י יְהֹוִֽה׃ ס
245,ב.ה.שׁנים.עד.שׁנת.ה.יבל,ב,1,?,"how to handle the second prep עד, here? Similar to הלאה. They could be treated as specifications as they probably are in the current dataset. This is not necessarily a bad encoding.",Leviticus 25:52,בַּשָּׁנִ֛ים עַד־שְׁנַ֥ת הַיֹּבֵ֖ל,691360,וְאִם־מְעַ֞ט נִשְׁאַ֧ר בַּשָּׁנִ֛ים עַד־שְׁנַ֥ת הַיֹּבֵ֖ל וְחִשַּׁב־לֹ֑ו כְּפִ֣י שָׁנָ֔יו יָשִׁ֖יב אֶת־גְּאֻלָּתֹֽו׃
252,ב.זמניהם,ב,1,?,"this does not appear adverbial but seems to be modifying ימי…; tell-tale sign is that it immediately follows a PP/NP, other questionable cases could be found in this way",Esther 9:31,בִּזְמַנֵּיהֶ֗ם,876172,לְקַיֵּ֡ם אֵת־יְמֵי֩ הַפֻּרִ֨ים הָאֵ֜לֶּה בִּזְמַנֵּיהֶ֗ם כַּאֲשֶׁר֩ קִיַּ֨ם עֲלֵיהֶ֜ם מָרְדֳּכַ֤י הַיְּהוּדִי֙ וְאֶסְתֵּ֣ר הַמַּלְכָּ֔ה וְכַאֲשֶׁ֛ר קִיְּמ֥וּ עַל־נַפְשָׁ֖ם וְעַל־זַרְעָ֑ם דִּבְרֵ֥י הַצֹּמֹ֖ות וְזַעֲקָתָֽם׃
651,ימים.רבים.שׁמונים.ו.מאת.יום,ימים,1,?,"this TP has quantified ימים with waw separating second part; this is typically represented as two heads, but not here. Why? Is it a problem?",Esther 1:4,יָמִ֣ים רַבִּ֔ים שְׁמֹונִ֥ים וּמְאַ֖ת יֹֽום׃,873951,בְּהַרְאֹתֹ֗ו אֶת־עֹ֨שֶׁר֙ כְּבֹ֣וד מַלְכוּתֹ֔ו וְאֶ֨ת־יְקָ֔ר תִּפְאֶ֖רֶת גְּדוּלָּתֹ֑ו יָמִ֣ים רַבִּ֔ים שְׁמֹונִ֥ים וּמְאַ֖ת יֹֽום׃
802,ל.עת.יום.ב.יום,ל.ב,1,?,There is more inconsistency here with the second prep specification. Is it or is it not a “head” element? This is done differently here than it is elsewhere…,1_Chronicles 12:23,לְעֶת־יֹ֣ום בְּיֹ֔ום,890356,כִּ֚י לְעֶת־יֹ֣ום בְּיֹ֔ום יָבֹ֥אוּ עַל־דָּוִ֖יד לְעָזְרֹ֑ו עַד־לְמַחֲנֶ֥ה גָדֹ֖ול כְּמַחֲנֵ֥ה אֱלֹהִֽים׃ פ
830,ל.רוח.ה.יום,ל,1,?,I wonder whether this really is temporal in nature? this needs more investigation. This case was marked as a statistically rare head term for TPs in the statistical TP head tests.,Genesis 3:8,לְר֣וּחַ הַיֹּ֑ום,652298,וַֽיִּשְׁמְע֞וּ אֶת־קֹ֨ול יְהוָ֧ה אֱלֹהִ֛ים מִתְהַלֵּ֥ךְ בַּגָּ֖ן לְר֣וּחַ הַיֹּ֑ום וַיִּתְחַבֵּ֨א הָֽאָדָ֜ם וְאִשְׁתֹּ֗ו מִפְּנֵי֙ יְהוָ֣ה אֱלֹהִ֔ים בְּתֹ֖וךְ עֵ֥ץ הַגָּֽן׃
864,מ.יום.אל.יום,מ,2,?,Is this different from coordinated phrases with two preps?,1_Chronicles 16:23,מִיֹּֽום־אֶל־יֹ֖ום,891227,שִׁ֤ירוּ לַֽיהוָה֙ כָּל־הָאָ֔רֶץ בַּשְּׂר֥וּ מִיֹּֽום־אֶל־יֹ֖ום יְשׁוּעָתֹֽו׃
876,מ.ימי.מלכי.אשׁור.עד.ה.יום.ה.זה,מ,1,?,"Only one prep counted due to no waw coordination. Have a look at two examples above, from Jer 36:2, where a near-verbatim phrase is deployed without the waw",Nehemiah 9:32,מִימֵי֙ מַלְכֵ֣י אַשּׁ֔וּר עַ֖ד הַיֹּ֥ום הַזֶּֽה׃,886316,וְעַתָּ֣ה אֱ֠לֹהֵינוּ הָאֵ֨ל הַגָּדֹ֜ול הַגִּבֹּ֣ור וְהַנֹּורָא֮ שֹׁומֵ֣ר הַבְּרִ֣ית וְהַחֶסֶד֒ אַל־יִמְעַ֣ט לְפָנֶ֡יךָ אֵ֣ת כָּל־הַתְּלָאָ֣ה אֲֽשֶׁר־מְ֠צָאַתְנוּ לִמְלָכֵ֨ינוּ לְשָׂרֵ֧ינוּ וּלְכֹהֲנֵ֛ינוּ וְלִנְבִיאֵ֥נוּ וְלַאֲבֹתֵ֖ינוּ וּלְכָל־עַמֶּ֑ךָ מִימֵי֙ מַלְכֵ֣י אַשּׁ֔וּר עַ֖ד הַיֹּ֥ום הַזֶּֽה׃
896,מ.נערי.עד.ה.יום.ה.זה,מ,1,?,"missing עד; I marked this case, but there are some other cases that I indeed didn’t mark…",1_Samuel 12:2,מִנְּעֻרַ֖י עַד־הַיֹּ֥ום הַזֶּֽה׃,735305,וְעַתָּ֞ה הִנֵּ֥ה הַמֶּ֣לֶךְ׀ מִתְהַלֵּ֣ךְ לִפְנֵיכֶ֗ם וַאֲנִי֙ זָקַ֣נְתִּי וָשַׂ֔בְתִּי וּבָנַ֖י הִנָּ֣ם אִתְּכֶ֑ם וַאֲנִי֙ הִתְהַלַּ֣כְתִּי לִפְנֵיכֶ֔ם מִנְּעֻרַ֖י עַד־הַיֹּ֥ום הַזֶּֽה׃


And for nheads...

In [12]:
nhead_anno[nhead_anno['mark']=='?']

,token,(n)heads_token,freq,mark,note,ex_ref,ex,ex_node,ex_verse
75,ב.דמי.ימי,דמי,1,?,"accents has a different division, excluding דמי; ESV translates as a TP. What is the basis?",Isaiah 38:10,בִּדְמִ֥י יָמַ֛י,782226,אֲנִ֣י אָמַ֗רְתִּי בִּדְמִ֥י יָמַ֛י אֵלֵ֖כָה בְּשַׁעֲרֵ֣י שְׁאֹ֑ול פֻּקַּ֖דְתִּי יֶ֥תֶר שְׁנֹותָֽי׃
84,ב.ה.חגים.ו.ב.ה.חדשׁים.ו.ב.ה.שׁבתות.ב.כל.מועדי.בית.ישׂראל,חגים.חדשׁים.שׁבתות,1,?,should מועדי be selected as a head too? Is it just an apposition?,Ezekiel 45:17,בַּחַגִּ֤ים וּבֶחֳדָשִׁים֙ וּבַשַּׁבָּתֹ֔ות בְּכָֽל־מֹועֲדֵ֖י בֵּ֣ית יִשְׂרָאֵ֑ל,821526,וְעַֽל־הַנָּשִׂ֣יא יִהְיֶ֗ה הָעֹולֹ֣ות וְהַמִּנְחָה֮ וְהַנֵּסֶךְ֒ בַּחַגִּ֤ים וּבֶחֳדָשִׁים֙ וּבַשַּׁבָּתֹ֔ות בְּכָֽל־מֹועֲדֵ֖י בֵּ֣ית יִשְׂרָאֵ֑ל הֽוּא־יַעֲשֶׂ֞ה אֶת־הַחַטָּ֣את וְאֶת־הַמִּנְחָ֗ה וְאֶת־הָֽעֹולָה֙ וְאֶת־הַשְּׁלָמִ֔ים לְכַפֵּ֖ר בְּעַ֥ד בֵּֽית־יִשְׂרָאֵֽל׃ ס
111,ב.ה.חיים.מספר.ימי.חיי.הבלו,חיים,1,?,this is difficult syntax; ESV also reads as a TP. But what exactly is happening in this TP?,Ecclesiastes 6:12,בַּֽחַיִּ֗ים מִסְפַּ֛ר יְמֵי־חַיֵּ֥י הֶבְלֹ֖ו,871283,כִּ֣י מִֽי־יֹודֵעַ֩ מַה־טֹּ֨וב לָֽאָדָ֜ם בַּֽחַיִּ֗ים מִסְפַּ֛ר יְמֵי־חַיֵּ֥י הֶבְלֹ֖ו וְיַעֲשֵׂ֣ם כַּצֵּ֑ל אֲשֶׁר֙ מִֽי־יַגִּ֣יד לָֽאָדָ֔ם מַה־יִּהְיֶ֥ה אַחֲרָ֖יו תַּ֥חַת הַשָּֽׁמֶשׁ׃
146,ב.ה.ליל,ליל,1,?,Why is this form recorded this way? without ה at end? The T.text rendition shows it properly…,Proverbs 31:18,בַלַּ֣יְלָה,867437,טָ֭עֲמָה כִּי־טֹ֣וב סַחְרָ֑הּ לֹֽא־יִכְבֶּ֖ה בַלַּ֣יְלָה נֵרָֽהּ׃
220,ב.ה.שׁנים.עד.שׁנת.ה.יבל,שׁנים,1,?,How to handle the עד cases? should be considrered here. As I noted in heads manual annotations.,Leviticus 25:52,בַּשָּׁנִ֛ים עַד־שְׁנַ֥ת הַיֹּבֵ֖ל,691360,וְאִם־מְעַ֞ט נִשְׁאַ֧ר בַּשָּׁנִ֛ים עַד־שְׁנַ֥ת הַיֹּבֵ֖ל וְחִשַּׁב־לֹ֑ו כְּפִ֣י שָׁנָ֔יו יָשִׁ֖יב אֶת־גְּאֻלָּתֹֽו׃
273,ב.יום.ו.עד.ה.יום.ה.זה,יום.יום,1,?,"what to do with coordinated version of this TP, when they’re not recorded in the same way without the waw coordination?",Jeremiah 11:7,בְּיֹום֩ וְעַד־הַיֹּ֣ום הַזֶּ֔ה,792928,כִּי֩ הָעֵ֨ד הַעִדֹ֜תִי בַּאֲבֹֽותֵיכֶ֗ם בְּיֹום֩ הַעֲלֹותִ֨י אֹותָ֜ם מֵאֶ֤רֶץ מִצְרַ֨יִם֙ וְעַד־הַיֹּ֣ום הַזֶּ֔ה הַשְׁכֵּ֥ם וְהָעֵ֖ד לֵאמֹ֑ר שִׁמְע֖וּ בְּקֹולִֽי׃
320,ב.ימי,ימי,4,?,cleft TP with missing infiinitive section,2_Chronicles 26:5,בִימֵי֙,901362,וַיְהִי֙ לִדְרֹ֣שׁ אֱלֹהִ֔ים בִּימֵ֣י זְכַרְיָ֔הוּ הַמֵּבִ֖ין בִּרְאֹ֣ת הָאֱלֹהִ֑ים וּבִימֵי֙ דָּרְשֹׁ֣ו אֶת־יְהוָ֔ה הִצְלִיחֹ֖ו הָאֱלֹהִֽים׃ ס
358,ב.ימי.קציר.ב.ה.ראשׁנים..תחלת.קציר.שׂערים,ימי,1,?,why is there an empty slot here? Also: note that this takes the CX of “in days of [person]”,2_Samuel 21:9,בִּימֵ֤י קָצִיר֙ בָּרִ֣אשֹׁנִ֔ים בִּתְחִלַּ֖ת קְצִ֥יר שְׂעֹרִֽים׃,752133,וַֽיִּתְּנֵ֞ם בְּיַ֣ד הַגִּבְעֹנִ֗ים וַיֹּקִיעֻ֤ם בָּהָר֙ לִפְנֵ֣י יְהוָ֔ה וַיִּפְּל֥וּ שְׁבַעְתָּ֖ם יָ֑חַד וְהֵ֨מָּה הֻמְת֜וּ בִּימֵ֤י קָצִיר֙ בָּרִ֣אשֹׁנִ֔ים בִּתְחִלַּ֖ת קְצִ֥יר שְׂעֹרִֽים׃
371,ב.יצקתו,יצקתו,1,?,"I may have marked this as “good” in heads manual search, but this needs to be scrutinzed more. Is this a temporal use?",1_Kings 7:24,בִּיצֻקָתֹֽו׃,756274,וּפְקָעִים֩ מִתַּ֨חַת לִשְׂפָתֹ֤ו׀ סָבִיב֙ סֹבְבִ֣ים אֹתֹ֔ו עֶ֚שֶׂר בָּֽאַמָּ֔ה מַקִּפִ֥ים אֶת־הַיָּ֖ם סָבִ֑יב שְׁנֵ֤י טוּרִים֙ הַפְּקָעִ֔ים יְצֻקִ֖ים בִּיצֻקָתֹֽו׃
398,ב.עד.יומם,עד,1,?,"Cf with 2 Sam 3:35, בעוד היום, where יום is selected instead of עוד, because עוד is considered a prepositional-ized lex here (or at least adjectival in nature). Perhaps both cases need to be reviewed.",Jeremiah 15:9,בְּעֹ֥ד יֹומָ֖ם,794172,אֻמְלְלָ֞ה יֹלֶ֣דֶת הַשִּׁבְעָ֗ה נָפְחָ֥ה נַפְשָׁ֛הּ בָּ֥א שִׁמְשָׁ֛הּ בְּעֹ֥ד יֹומָ֖ם בֹּ֣ושָׁה וְחָפֵ֑רָה וּשְׁאֵֽרִיתָ֗ם לַחֶ֧רֶב אֶתֵּ֛ן לִפְנֵ֥י אֹיְבֵיהֶ֖ם נְאֻם־יְהוָֽה׃ ס


# Conclusions

This analysis manually looked over 1164 head to token phrase mappings and 888 nhead to token mappings. Many of the problematic cases have been fixed by exporting new phrase functions, adjusting the heads pipeline, or planning for new chunk objects that can handle complexity above/below the phrase level. Some more complex cases with heads still need to be adjusted (see bold notes in discussion on fixes above).